In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [ ]:
# Create a set of random latitude and longitude combinations.
# I think -90 to 90 is too likely to pick a non populated area
# Even using -60 to 70 you are going to get some extremely remote locations, most over water 
maxsize = 1500
lats = np.random.uniform(low=-60.000, high=70.000, size=maxsize)
lngs = np.random.uniform(low=-180.000, high=180.000, size=maxsize)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
newlats = list(lat_lngs)
newlats
#for lat_lng in lat_lngs:
#    print(lat_lng[0], lat_lng[1])

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = newlats
len(coordinates)

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
cities

In [ ]:
# okay so we have more than 500 cities but should we eliminate some with low populations? can we get population data? 
#I already know the nearest_city function only shows cities with population > 500 people
len(cities)